In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
dataset = get_file(fname="horse-or-human.zip", origin=url, extract=True, cache_dir=".")

gene = ImageDataGenerator(rotation_range=0.2,
                          zoom_range=0.2,
                          horizontal_flip=True,
                          validation_split=0.2,
                          preprocessing_function=preprocess_input)

In [ ]:
train_data = gene.flow_from_directory(directory="datasets",
                                      target_size=(256, 256),
                                      class_mode="categorical",
                                      shuffle=True,
                                      seed=38,
                                      subset="training")

test_data = gene.flow_from_directory(directory="datasets",
                                     target_size=(256, 256),
                                     class_mode="categorical",
                                     shuffle=True,
                                     seed=38,
                                     subset="validation")

Found 822 images belonging to 2 classes.
Found 205 images belonging to 2 classes.


In [ ]:
vgg16 = VGG16(include_top=False, weights="imagenet", input_shape=(256, 256, 3))
vgg16.trainable = False

In [ ]:
model = Sequential()

model.add(vgg16)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(2, activation="softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(train_data, epochs=5)

Epoch 1/5
26/26 [==============================] - 18s 615ms/step - loss: 0.0848 - accuracy: 0.9635
Epoch 2/5
26/26 [==============================] - 16s 612ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 3/5
26/26 [==============================] - 16s 627ms/step - loss: 3.9800e-04 - accuracy: 1.0000
Epoch 4/5
26/26 [==============================] - 16s 603ms/step - loss: 0.0106 - accuracy: 0.9964
Epoch 5/5
26/26 [==============================] - 16s 616ms/step - loss: 2.3796e-04 - accuracy: 1.0000


In [ ]:
model.evaluate(test_data)

7/7 [==============================] - 4s 517ms/step - loss: 0.0010 - accuracy: 1.0000


[0.0010384464403614402, 1.0]

In [ ]:
images = []

for directory in os.listdir("datasets"):
  for file in os.listdir(os.path.join("datasets", directory)):
    images.append(os.path.join("datasets", directory, file))

In [ ]:
images = [tf.keras.utils.img_to_array(tf.keras.utils.load_img(img)) for img in images]

In [ ]:
images = tf.data.Dataset.from_tensor_slices(images)